In [1]:
import numpy as np
from PIL import Image
from matplotlib import pyplot as plt

from keras.preprocessing.image import load_img, img_to_array
from scipy.optimize import fmin_l_bfgs_b 
from keras.applications import vgg19
from keras import backend as K
from keras.preprocessing.image import save_img

import tensorflow as tf

tf.compat.v1.disable_eager_execution() 

In [2]:
def preprocess_image(image_path, resized_width, resized_height):
    img = load_img(image_path, target_size=(resized_width, resized_height))
    img = img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img = vgg19.preprocess_input(img)
    return img

def deprocess_image(x, resized_width, resized_height):
    x = x.reshape((resized_width, resized_height, 3))

    # Remove zero-center by mean pixel. Necessary when working with VGG model
    x[:, :, 0] += 103.939
    x[:, :, 1] += 116.779
    x[:, :, 2] += 123.68

    # Format BGR->RGB
    x = x[:, :, ::-1]
    x = np.clip(x, 0, 255).astype('uint8')
    return x

def save(filename, generated):
    save_img(filename, Image.fromarray(generated))

In [3]:
def gram_matrix(x):
    features = K.batch_flatten(K.permute_dimensions(x, (2, 0, 1)))
    gram = K.dot(features, K.transpose(features))
    return gram


In [4]:
def style_loss_per_layer(style, combination, resized_width, resized_height):
    S = gram_matrix(style)
    C = gram_matrix(combination)
    channels = 3
    size = resized_width * resized_height
    return K.sum(K.square(S - C)) / (4. * (channels ** 2) * (size ** 2))


def total_style_loss(feature_layers, outputs_dict, resized_width, resized_height, style_weight):
    loss = K.variable(0.)
    for layer_name in feature_layers:
        layer_features = outputs_dict[layer_name]
        style_reference_features = layer_features[1, :, :, :]
        combination_features = layer_features[2, :, :, :]
        sl = style_loss_per_layer(style_reference_features, combination_features, resized_width, resized_height)
        loss = loss + (style_weight / len(feature_layers)) * sl
    return loss


def content_loss(layer_features):
    base_image_features = layer_features[0, :, :, :]
    combination_features = layer_features[2, :, :, :]
    return K.sum(K.square(combination_features - base_image_features))


def total_variation_loss(x, resized_width, resized_height):
    a = K.square(x[:, :resized_width - 1, :resized_height - 1, :] - x[:, 1:, :resized_height - 1, :])
    b = K.square(x[:, :resized_width - 1, :resized_height - 1, :] - x[:, :resized_width - 1, 1:, :])
    return K.sum(K.pow(a + b, 1.25))


def total_loss(outputs_dict, content_weight, resized_width, resized_height, style_weight, total_variation_weight, combination_image):
    loss = K.variable(0.)

    feature_layers_content = outputs_dict['block5_conv2']
    loss = loss + content_weight * content_loss(feature_layers_content)

    feature_layers_style = ['block1_conv1', 'block2_conv1',
                            'block3_conv1', 'block4_conv1',
                            'block5_conv1']
    loss = loss + total_style_loss(feature_layers_style, outputs_dict, resized_width, resized_height, style_weight) * style_weight

    loss = loss + total_variation_weight * total_variation_loss(combination_image, resized_width, resized_height)
    
    return loss

In [5]:
def eval_loss_and_grads(x, resized_width, resized_height, f_outputs):
    x = x.reshape((1, resized_width, resized_height, 3))
    outs = f_outputs([x])
    loss_value = outs[0]
    if len(outs[1:]) == 1:
        grad_values = outs[1].flatten().astype('float64')
    else:
        grad_values = np.array(outs[1:]).flatten().astype('float64')
    return loss_value, grad_values

class Evaluator(object):
    def __init__(self, resized_width, resized_height, f_outputs):
        self.loss_value = None
        self.grads_values = None
        self.resized_width = resized_width
        self.resized_height = resized_height
        self.f_outputs = f_outputs
        
    def loss(self, x):
        assert self.loss_value is None
        loss_value, grad_values = eval_loss_and_grads(x, self.resized_width, self.resized_height, self.f_outputs)
        self.loss_value = loss_value
        self.grad_values = grad_values
        return self.loss_value

    def grads(self, x):
        assert self.loss_value is not None
        grad_values = np.copy(self.grad_values)
        self.loss_value = None
        self.grad_values = None
        return grad_values


In [9]:
def run(content_path, style_path, itr, filename):
    # Variables declaration
    base_image_path = content_path
    style_reference_image_path = style_path
    iterations = itr
    
    # Weights to compute the final loss
    total_variation_weight = 0
    style_weight = 2
    content_weight = 5
    
    # Dimensions of the generated picture.
    width, height = load_img(base_image_path).size
    resized_width = 512
    resized_height = int(width * resized_width / height)
    
    # Get tensor representations of our images
    base_image = K.variable(preprocess_image(base_image_path, resized_width, resized_height))
    style_reference_image = K.variable(preprocess_image(style_reference_image_path, resized_width, resized_height))
    
    # Placeholder for generated image
    combination_image = K.placeholder((1, resized_width, resized_height, 3))
    
    # Combine the 3 images into a single Keras tensor
    input_tensor = K.concatenate([base_image,
                                  style_reference_image,
                                  combination_image], axis=0)
    
    # Build the VGG19 network with our 3 images as input
    # the model is loaded with pre-trained ImageNet weights
    model = vgg19.VGG19(input_tensor=input_tensor,weights='imagenet', include_top=False)
    
    # Get the outputs of each key layer, through unique names.
    outputs_dict = dict([(layer.name, layer.output) for layer in model.layers])
    loss = total_loss(outputs_dict, content_weight, resized_width, resized_height, style_weight, total_variation_weight, combination_image)
    
    # Get the gradients of the generated image
    grads = K.gradients(loss, combination_image)
    outputs = [loss]
    outputs += grads
    
    f_outputs = K.function([combination_image], outputs)
    
    evaluator = Evaluator(resized_width, resized_height, f_outputs)

    x = preprocess_image(base_image_path, resized_width, resized_height)
    
    # The oprimizer is fmin_l_bfgs
    for i in range(iterations):
        print('Iteration: ', i)
        x, min_val, info = fmin_l_bfgs_b(evaluator.loss,
                                         x.flatten(),
                                         fprime=evaluator.grads,
                                         maxfun=25)
    
        print('Current loss value:', min_val)
    
        # Save current generated image
        img = deprocess_image(x.copy(), resized_width, resized_height)
        fname = str(i) + '.png'
    plt.figure()
    plt.title("Generated Image")
    plt.imshow(img)
    save(filename, img)

In [10]:
if __name__ == '__main__':
    
    # Change the file path and the iteration value as desired
    filename = "people2.jpg"
    content_path = "../input/style-transfer-files/Content/people2.jpg"
    style_path = "../input/style-transfer-files/Style/1.jpg"
    itr = 5
    
    run( content_path, style_path, itr, filename)
    
    filename = "city0.jpg"
    content_path = "../input/style-transfer-files/Content/city0.jpg"
    style_path = "../input/style-transfer-files/Style/0.jpg"
    itr = 5
    
    run( content_path, style_path, itr, filename)